In [1]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2
import pg8000

from tkinter import Tk
from tkinter.filedialog import askdirectory, askopenfilename

from tqdm import tqdm

# Setup PostgreSQL connection

In [12]:
ev_local = {
    'user': 'evan',
    'password': 'Discretionary@020',
    'host': 'localhost',
    'port': 5432,
    'database': 'postgis_cookbook'
}

print(ev_local)

{'user': 'evan', 'password': 'Discretionary@020', 'host': 'localhost', 'port': 5432, 'database': 'postgis_cookbook'}


In [13]:
# setup connection engine

engine = create_engine(str(r"postgresql+psycopg2://%s:" + '%s' + "@%s:%s/%s")
                           % (ev_local['user'], ev_local['password'], ev_local['host'], ev_local['port'], ev_local['database']),
                       encoding='latin1')


# Function import csv to PostGis

In [4]:
#https://stackoverflow.com/questions/39494056/progress-bar-for-pandas-dataframe-to-sql

def chunker(seq, size):
    # from http://stackoverflow.com/a/434328
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

In [5]:
def csv2points(df,engine,x,y,table_name,schema,**kwargs):
    
    chunksize = int(len(df) / 10) # 1%
    
    with tqdm(total=len(df)) as pbar:
        for i, cdf in enumerate(chunker(df, chunksize)):
            replace = "replace" if i == 0 else "append"
            cdf.to_sql(name= table_name, con=engine, schema = schema, if_exists=replace, index=False)
#             df.to_sql(name= table_name, con=engine, schema = schema, if_exists='replace', index=False)
            pbar.update(chunksize)
    
    if bool(kwargs) == True:   
        add_pk=  'ALTER TABLE ' + schema + '.'+ table_name + ' ADD PRIMARY KEY ('+  kwargs['pk'] + ');'
        print (add_pk)
        
        add_constraint = 'ALTER TABLE ' + schema + '.'+ table_name + ' ADD CONSTRAINT pk PRIMARY KEY (' + kwargs['pk'] + ');'
        print (add_constraint)
        
        with engine.connect() as con:
#             con.execute(add_pk)
            con.execute(add_constraint)

    # add column with geometry type
    add_col = 'ALTER TABLE ' + schema + '.'+ table_name + ' ADD COLUMN geo_loc geometry(POINT, 4326);'

    # Import the points in the geometric column using the ST_MakePoint
#     update_col = 'UPDATE ' + schema + '.'+ table_name + ' SET geo_loc = ST_SetSRID(ST_MakePoint(SiteLongitude_WGS84,SiteLatitude_WGS84), 4326);'
    update_col = 'UPDATE ' + schema + '.'+ table_name + ' SET geo_loc = ST_SetSRID(ST_MakePoint(' + x +',' + y + '), 4326);'

    # create a spatial index for the geometric column of the table: 
    idx_col = 'CREATE INDEX idx_' + table_name + ' ON ' + schema + '.'+ table_name +' USING GIST (geo_loc)';  
#     print (idx_col)

    with engine.connect() as con:
        con.execute(add_col)
        con.execute(update_col)
        con.execute(idx_col)
        con.close()
    print('Complete!')


# Load firenews.csv Dataset

In [14]:
root = Tk()
root.withdraw()
data = askopenfilename(parent=root, title='Select the csv file')

df = pd.read_csv(data)
df.head()

,x,y,place,size,update,startdate,enddate,title,url
0,2.209300,44.591830,Boisse-Penchot,50.0,2010/08/11,2010/08/10,2010/08/10,50 hectares ravagés par les flammes,http://www.ladepeche.fr/article/2010/08/11/886...
1,-6.814122,41.839924,Bragança,NaN,2011/07/30,2011/07/29,2011/07/29,Incêndios: Dominado fogo de Bragança - Protecç...,http://aeiou.expresso.pt/incendios-dominado-fo...
2,-6.814122,41.839924,Bragança,NaN,2011/07/30,2011/07/29,2011/07/29,Incêndios: 30 bombeiros combatem chamas em Bra...,http://www.dn.pt/Inicio/interior.aspx?content_...
3,27.201504,38.442517,Izmir,NaN,2011/08/16,2011/08/15,2011/08/15,İzmir'deki Orman Yangınları - Son Dakika,http://www.sondakika.com/haber-izmir-deki-orma...
4,23.648596,35.495426,Kissamos,NaN,2012/07/19,2012/07/18,2012/07/19,Χανιά: Σε ύφεση η φωτιά στην Κίσαμο - Εξπρές,http://www.express.gr/news/ellada/628375oz_201...


In [15]:
csv2points(df,engine,'x','y','firenews','chp01')

3300it [00:00, 6666.82it/s]                          

Complete!


# Load global_24h.csv Dataset

In [6]:
root = Tk()
root.withdraw()
data = askopenfilename(parent=root, title='Select the csv file')

In [7]:
df = pd.read_csv(data)
df.head()

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,confidence,version,bright_t31,frp
0,-23.386,-46.197,307.5,1.1,1.0,2012-08-20,140,T,54,5.0,285.7,16.5
1,-22.952,-47.574,330.1,1.2,1.1,2012-08-20,140,T,100,5.0,285.2,53.9
2,-23.726,-56.108,333.3,4.7,2.0,2012-08-20,140,T,100,5.0,283.5,404.1
3,-23.729,-56.155,311.8,4.7,2.0,2012-08-20,140,T,61,5.0,272.0,143.1
4,-23.708,-56.110,335.7,4.7,2.0,2012-08-20,140,T,100,5.0,285.2,439.5


In [11]:
csv2points(df,engine,'longitude','latitude','global_24h','chp01')

33352it [00:04, 7238.76it/s]                           


Complete!
